Analyze the Vine DataFrame and determine if the Vine reviews are biased.

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [90.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [903 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports In

In [2]:


from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").getOrCreate()



In [3]:

from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

video_games_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [4]:



dropna_df = video_games_df.dropna()
clean_df = dropna_df.drop_duplicates()
clean_df.show(10)



+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   20009611|R100E0YCS06YNR|B00EFFW0HC|     277685961|       Battlefield 4|     Video Games|          5|            0|          0|   N|                Y|          Five Stars|     great thank you|2015-07-04 00:00:00|
|         US|   21036809|R100HR44BSB9I4|B007FTE2VW|     265303108|SimCity - Limited...| 

In [5]:
vine_df = clean_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100E0YCS06YNR|          5|            0|          0|   N|                Y|
|R100HR44BSB9I4|          1|            1|          1|   N|                Y|
|R100UABZOUF1JP|          1|            0|          0|   N|                Y|
|R101MOW8A5WC4J|          5|            0|          0|   N|                Y|
|R101X7KSBGIWF1|          1|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [6]:
df2 = vine_df.filter(vine_df["total_votes"] >= 20)
df2.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R23Z2K6GIQ2E6O|          4|           19|         20|   N|                Y|
|R2SSU7RT61AYK2|          4|           87|         98|   N|                Y|
| RPRP6F6ABCLLK|          2|           28|         39|   N|                Y|
| RGOVXI764T1KO|          1|            5|         23|   N|                Y|
|R39UPDNM5TECB5|          5|           23|         23|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [7]:
df3 = df2.filter(df2["helpful_votes"]/df2["total_votes"] >= 0.5)
df3.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R23Z2K6GIQ2E6O|          4|           19|         20|   N|                Y|
|R2SSU7RT61AYK2|          4|           87|         98|   N|                Y|
| RPRP6F6ABCLLK|          2|           28|         39|   N|                Y|
|R39UPDNM5TECB5|          5|           23|         23|   N|                N|
|R2RW4T8KWXQXZQ|          1|           40|         65|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [8]:
vine_yes_df = df3.filter(df3["vine"] == "Y")
vine_yes_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1PU7JY8XL5M1N|          5|           19|         24|   Y|                N|
|R23WY68NT94WKE|          5|          125|        143|   Y|                N|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|
|R1UBDJF7PWB8BL|          2|           43|         59|   Y|                N|
|R1TNOMT3O5DZ0Z|          4|           27|         32|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [9]:
from pyspark.sql.functions import col, avg

In [10]:
vine_yes_df.describe().show()

+-------+--------------+------------------+-----------------+------------------+----+-----------------+
|summary|     review_id|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+--------------+------------------+-----------------+------------------+----+-----------------+
|  count|            94|                94|               94|                94|  94|               94|
|   mean|          null| 4.202127659574468|54.59574468085106|61.787234042553195|null|             null|
| stddev|          null|0.9791348741656417|65.26098459822538| 68.90976994895394|null|             null|
|    min|R10FO5UKKVZBK2|                 1|               14|                20|   Y|                N|
|    max| RYBLWBAL5M1AY|                 5|              347|               362|   Y|                N|
+-------+--------------+------------------+-----------------+------------------+----+-----------------+



In [11]:
vine_no_df = df3.filter(df3["vine"] == "N")
vine_no_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R23Z2K6GIQ2E6O|          4|           19|         20|   N|                Y|
|R2SSU7RT61AYK2|          4|           87|         98|   N|                Y|
| RPRP6F6ABCLLK|          2|           28|         39|   N|                Y|
|R39UPDNM5TECB5|          5|           23|         23|   N|                N|
|R2RW4T8KWXQXZQ|          1|           40|         65|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [13]:
paid_five_star_review = vine_yes_df[vine_yes_df['star_rating']== 5].count()
paid_review = vine_yes_df.count()
percentage_vine = float(paid_five_star_review) / float(paid_review)

print(f"total paid reviews: {paid_review}")
print(f"paid reviewss with 5-stars: {paid_five_star_review}")
print(f"% of paid 5-star reviews: {percentage_vine}")

total paid reviews: 94
paid reviewss with 5-stars: 48
% of paid 5-star reviews: 0.5106382978723404


In [14]:
unpaid_five_star_reviews = vine_no_df[vine_no_df['star_rating']== 5].count()
unpaid_reviews = vine_no_df.count()
unpaid_percentage_5_star = float(unpaid_five_star_reviews) / float(unpaid_reviews)

print(f"total of unpaid reviews: {unpaid_reviews}")
print(f"unpaid 5-star reviews: {unpaid_five_star_reviews}")
print(f"% of unpaid 5-star reviews: {unpaid_percentage_5_star}")

total of unpaid reviews: 40471
unpaid 5-star reviews: 15663
% of unpaid 5-star reviews: 0.38701786464381904
